In [142]:
!pip install textblob

In [143]:
import json
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
from textblob import TextBlob

#Importing libraries required for the case study and to plot charts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

#Importing Regular Expressions for String manipulation
import re

# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [144]:
import json
import pandas as pd

with open('/content/complaints-2021-05-14_08_16.json', encoding='latin-1') as j:
data = json.load(j)

df = pd.json_normalize

IndentationError: expected an indented block after 'with' statement on line 7 (<ipython-input-144-ad611ec10058>, line 8)

In [ ]:
# Inspect the dataframe to understand the given data.
df.head()

In [ ]:
#shape of dataframe
df.shape

In [ ]:
#print the column names
df.columns

In [ ]:
#Assign new column names
df = df[['_source.complaint_what_happened','_source.product','_source.sub_product',]]
df.head()

In [ ]:
# lets rename the column names to be more read friendly

df = df.rename(columns={'_source.complaint_what_happened': 'complaint_text', '_source.product': 'category','_source.sub_product': 'sub_category'})
df.head()

In [ ]:
# lets merge the category and sub-category this will help us in deciding the topics after NMF modelling
df['category'] = df['category'] + '+' + df['sub_category']
df = df.drop(['sub_category'],axis= 1)
df.head()

In [ ]:
# lets check the nan complaints
df.complaint_text.isnull().sum()

In [ ]:
# lets check the empty string complaints
len(df[df['complaint_text']==''])

In [ ]:
#Assign nan in place of blanks in the complaints column
# lets replace empty complain with nan value
df[df['complaint_text']==''] = np.nan
df.complaint_text.isnull().sum()

In [ ]:
#Remove all rows where complaints column is nan
# Lets drop all the rows where complaint_text is null
df = df[~df['complaint_text'].isnull()]
df.complaint_text.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_texts(text):
    #Make the text lowercase
    text=text.lower()

    #Remove text in square brackets
    text=re.sub(r'\[.*?\]','',text)

    #Remove punctuation
    text=re.sub(r'[%s]%re.escape(string.punctuation)','',text)

    #Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    return text

In [ ]:
#Cleaning df['complaint_what_happened']
df['complaint_text']= df['complaint_text'].apply(lambda x: clean_texts(x))

In [ ]:
df.head()

In [ ]:
#Write your function to Lemmatize the texts
def lemma_texts(text):

    # Initialize empty list to store lemmas
    lemma_list = []

    # Extract lemmas of given text and add to the list 'sent'
    document = nlp(text)
    for word in document:
        lemma_list.append(word.lemma_)

    # return string converted form of the list of lemmas
    return " ".join(lemma_list)

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def lemma_texts_batch(texts):
    """Lemmatizes a batch of texts using spaCy's nlp.pipe for efficiency."""
    docs = nlp.pipe(texts, disable=["parser", "ner"])  # Disable unnecessary components
    return [" ".join([token.lemma_ for token in doc]) for doc in docs]

df["lemmatized_complaint"] = lemma_texts_batch(df["complaint_text"].tolist())

In [ ]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints
df_clean=df[['complaint_text','lemmatized_complaint']]

In [ ]:
df_clean.head()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
#Write your function to extract the POS tags

# Extracting singular nouns
def singular_nouns(text):


    # Creating a textblob object
    text_blob = TextBlob(text)

    # extracting words with tags 'NN', joining them and return
    return ' '.join([ word for (word,tag) in text_blob.tags if tag == "NN"])


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
df_clean["complaint_POS_removed"] = df_clean.apply(lambda x: singular_nouns(x['lemmatized_complaint']), axis=1)

# View the dataframe
df_clean.head()

## 5. Exploratory Data Analysis

In [ ]:
#lenght of character in 'complaint_POS_removed'
char_len=[len(x) for x in df_clean['complaint_POS_removed']]
char_len[:15]

In [ ]:
## Write your code here to visualise the data according to the 'Complaint' character length
plt.figure(figsize=[10,6])
sns.histplot(data = char_len,bins=55)
plt.title('Distribution of Complaint Character Length', fontsize=25)
plt.xlabel('Complaint Character Length',size=20)
plt.ylabel('No. of Complaints',size=20)

In [ ]:
!pip install wordcloud

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text
from wordcloud import WordCloud, STOPWORDS

stop_words = set(STOPWORDS)
word_cloud = WordCloud(
                          background_color='green',
                          stopwords=stop_words,
                          max_font_size=38,
                          max_words=38,
                          random_state=42
                         ).generate(str(df_clean['complaint_POS_removed']))

fig = plt.figure(figsize=(20,16))
plt.imshow(word_cloud)
plt.axis('off')
plt.show()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [ ]:
def get_top_n_bigram(text, ngram=1, top=None):
    vec = CountVectorizer(ngram_range=(ngram, ngram), stop_words='english').fit(text)
    bag_of_words = vec.transform(text)

    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top]

In [ ]:
top_30_unigrams = get_top_n_bigram(df_clean.complaint_POS_removed, ngram=1, top=30)
top_30_bigrams = get_top_n_bigram(df_clean.complaint_POS_removed, ngram=2, top=30)
top_30_trigrams = get_top_n_bigram(df_clean.complaint_POS_removed, ngram=3, top=30)

In [ ]:
#Print the top 10 words in the unigram frequency
print('Top 10 unigrams:\n')

df_unigram = pd.DataFrame(top_30_unigrams, columns = ['unigram' , 'count'])
df_unigram.head(10)

In [ ]:
# Plot the top 30 unigrams
plt.figure(figsize=(12,6))
sns.barplot(x='unigram', y='count', data=df_unigram, palette="Blues_d")
plt.xticks(rotation=90)
plt.title("Top 30 unigrams in the Complaint text after removing stop words and lemmatization", fontsize=20)
plt.show()

In [ ]:
#Print the top 10 words in the bigram frequency
print('Top 10 bigrams:\n')

df_bigram = pd.DataFrame(top_30_bigrams, columns = ['bigram' , 'count'])
df_bigram.head(10)

In [ ]:
# Plot the top 30 bigrams
plt.figure(figsize=(12,6))
sns.barplot(x='bigram', y='count', data=df_bigram, palette="Blues_d")
plt.xticks(rotation=90)
plt.title("Top 30 bigrams in the Complaint text after removing stop words and lemmatization", fontsize=20)
plt.show()

In [ ]:
#Print the top 10 words in the trigram frequency
print('Top 10 trigrams:\n')

df_trigram = pd.DataFrame(top_30_trigrams, columns = ['trigram' , 'count'])
df_trigram.head(10)

In [ ]:
# Plot the top 30 trigrams
plt.figure(figsize=(15,6))
sns.barplot(x='trigram', y='count', data=df_trigram, palette="Blues_d")
plt.xticks(rotation=90)
plt.title("Top 30 trigrams in the Complaint text after removing stop words and lemmatization", fontsize=20)
plt.show()

In [ ]:
df_clean['complaint_POS_removed'] = df_clean['complaint_POS_removed'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean.head()

## 6. Feature Extraction

In [ ]:
#Write your code here to initialise the TfidfVectorizer
tfidf = TfidfVectorizer(min_df=2, max_df=0.95, stop_words='english')

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.
dtm = tfidf.fit_transform(df_clean.complaint_POS_removed)

In [ ]:
dtm

In [ ]:
# Use get_feature_names_out() instead of get_feature_names()
tfidf.get_feature_names_out()[:10]

## 7. Topic Modelling using NMF (Non-Matrix Factorization)

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics =  5

#keep the random_state =40
nmf_model = NMF(n_components=num_topics, random_state=40)

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names_out())

In [ ]:
# Print the top word of a sample component
topic_single = nmf_model.components_[0]
topic_single.argsort()
top_word_index = topic_single.argsort()[-20:]
for index in top_word_index:
    print(tfidf.get_feature_names_out()[index])

In [ ]:
#Print the Top20 words for each of the topics
for index, topic in enumerate(nmf_model.components_):
    print(f'TOP 20 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-20:]])
    print('\n')

In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4

topic_result = nmf_model.transform(dtm)
topic_result[0].round(2)
topic_result[0].argmax()
topic_result.argmax(axis=1)

In [ ]:
# Create a new 'Topic' column and assign the best topic to each of the complaints

df_clean['Topic'] = topic_result.argmax(axis=1)

In [ ]:
df_clean.shape

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean_5=df_clean.groupby('Topic').head(5)
df_clean_5.sort_values('Topic')

## 8. Mapping Relevant Topic to Each Complaint

In [ ]:
#Create the dictionary of Topic names and Topics
topic_mapping = {
    0: 'Bank Account services',
    1: 'Credit card or prepaid card',
    2: 'Others',
    3: 'Theft/Dispute Reporting',
    4: 'Mortgage/Loan'
}

#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(topic_mapping)

In [ ]:
df_clean.head()

In [ ]:
# Plot graph to check class imbalance
plt.figure(figsize=(12,6))
sns.countplot(x='Topic', data=df_clean, color = 'Green')

## 9. Build the Supervised Model for making a prediction for new complaint

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data

training_data = df_clean[['complaint_text','Topic']]

In [ ]:
training_data.head()

In [ ]:
training_data['complaint_text'] = training_data['complaint_text'].str.replace('xxxx','')
training_data.head()

In [ ]:
# View value counts of the five topics
training_data['Topic'].value_counts()

## 10. Data Preprocessing

In [ ]:
## Reverse topic names mapping for supervised learning

reverse_topic_mapping = {
    'Bank Account services' :0,
    'Credit card or prepaid card':1,
    'Others':2,
    'Theft/Dispute Reporting':3,
    'Mortgage/Loan':4
}
#Replace Topics with Topic Names
training_data['Topic'] = training_data['Topic'].map(reverse_topic_mapping)
training_data.head()

In [ ]:

#Write your code to get the Vector count
count_vector=CountVectorizer()

#Write your code here to transform the word vector to tf-idf
X_train_count=count_vector.fit_transform(training_data['complaint_text'])

In [ ]:
# Saving Word Vector on disk for later usage
import pickle

pickle.dump(count_vector.vocabulary_, open("/content/complaints-2021-05-14_08_16.json","wb"))

In [ ]:
# Transform the word vector to tf-idf
tfidf_transform= TfidfTransformer()
X_train_tfidf = tfidf_transform.fit_transform(X_train_count)

In [ ]:
# Save TF-IDF on disk for later usage
pickle.dump(tfidf_transform, open("/content/complaints-2021-05-14_08_16.json","wb"))

In [ ]:
# Train-Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.Topic, test_size=0.20, random_state=42)

print(f"X_train Shape: {X_train.shape}")
print(f"y_train Shape: {y_train.shape}")
print(f"X_test Shape: {X_test.shape}")
print(f"y_test Shape: {y_test.shape}")

## 11. Model Building & Evaluation

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# Create a function to evaluate models
def model_eval(y_test, y_pred, model_name):

    # print classification report of classifier
    print(f"CLASSIFICATION REPORT for {model_name}\n")
    print(classification_report(y_test, y_pred, target_names=["Bank Account services", "Credit card or prepaid card", "Others", "Theft/Dispute Reporting",
"Mortgage/Loan"]))

    # plot confusion matrix of the classifier
    plt.figure(figsize=(10,7))
    plt.title(f"CONFUSION MATRIX for {model_name}\n")
    matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(matrix, annot=True, cbar=None, cmap="Greens", fmt='d', xticklabels=["Bank Account services", "Credit card or prepaid card", "Others", "Theft/Dispute Reporting",
"Mortgage/Loan"], yticklabels=["Bank Account services", "Credit card or prepaid card", "Others", "Theft/Dispute Reporting",
"Mortgage/Loan"])
    plt.show()

    return

### **Model 1. Naive Bayes**

In [ ]:
# Import required library
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Run the Multinomial Naive Bayes with default parameters

model_name = 'NAIVE BAYES'
nb = MultinomialNB()
nb.fit(X_train, y_train)

In [ ]:
y_pred_nb = nb.predict(X_test)

In [ ]:
# Calculate F1 Score of model using weighted average method
f1_nb = f1_score(y_test, y_pred_nb, average="weighted")
f1_nb

In [ ]:
# Evaluate the Naive Bayes classifier
model_eval(y_test, y_pred_nb, model_name)

### **Model 2. Logistic Regression**

In [ ]:
# Import required library
from sklearn.linear_model import LogisticRegression

In [ ]:
# Run the Logistic Regression model with default parameters
model_name = 'LOGISTIC REGRESSION'
lr = LogisticRegression()
lr.fit(X_train, y_train)


In [ ]:
# Making Prediction with test data
y_pred_lr = lr.predict(X_test)

In [ ]:
# Calculate F1 Score of tuned model using weighted average method
f1_lr = f1_score(y_test, y_pred_lr, average="weighted")
f1_lr

In [ ]:
# Evaluate the tuned Logistic Regression classifier
model_eval(y_test, y_pred_lr, model_name)

### **Model 3. Decision Tree**

In [ ]:
# Import required library
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Run Decision Tree with default hyperparameters
model_name = 'DECISION TREE'
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)


In [ ]:
y_pred_dt =dt.predict(X_test)

In [ ]:
# Calculate F1 Score of tuned model using weighted average method
f1_dt = f1_score(y_test, y_pred_dt, average="weighted")
f1_dt

In [ ]:
# Evaluate the tuned Decision Tree classifier
model_eval(y_test, y_pred_dt, model_name)

### **Model 4. Random Forest**

In [ ]:
# Import required library
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Run Decision Tree with default hyperparameters
model_name = 'RANDOM FOREST'
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


In [ ]:
# Making Prediction on Test Data

y_pred_rf =rf.predict(X_test)

In [ ]:
# Calculate F1 Score of tuned model using weighted average method
f1_rf = f1_score(y_test, y_pred_rf, average="weighted")
f1_rf

In [ ]:
# Evaluate the tuned Decision Tree classifier
model_eval(y_test, y_pred_rf, model_name)

## **Predicting Category of New Complaint via Logistic Regression on custom text**


In [ ]:
# Save Logistic Regression model as pickle file in device
pickle.dump(lr, open("/content/complaints-2021-05-14_08_16.json", "wb"))

In [ ]:
# raw complaint text
test_complaint= 'I tried to make a transaction at a supermarket retail store, using my chase \
debit/atm card, but the transaction was declined. I am still able to withdraw money out of an \
ATM machine using the same debit card. Please resolve this issue.'

In [ ]:
# vectorize and tf-idf tranform
test = count_vector.transform([test_complaint])
test_tfidf = tfidf_transform.transform(test)

In [ ]:
# predict
prediction=lr.predict(test_tfidf)
prediction

In [ ]:
topic_mapping[prediction[0]]

## **Create a list of Custom Complaints and then Predict the Category of Each Complaint**

In [ ]:
# Function to predict a topic for custom text

def topic_predictor(text):

    target_names = ["Bank Account services", "Credit card or prepaid card", "Others", "Theft/Dispute Reporting", "Mortgage/Loan"]

    load_vec = CountVectorizer(vocabulary=pickle.load(open("/content/drive/MyDrive/Case Study NLP - Automatic Ticket Classification/count_vector.pkl", "rb")))
    load_tfidf = pickle.load(open("/content/drive/MyDrive/Case Study NLP - Automatic Ticket Classification/tfidf.pkl","rb"))
    load_model = pickle.load(open("/content/drive/MyDrive/Case Study NLP - Automatic Ticket Classification/logreg_model.pkl","rb"))

    X_new_count = load_vec.transform(text)
    X_new_tfidf = load_tfidf.transform(X_new_count)
    prediction = load_model.predict(X_new_tfidf)

    return target_names[prediction[0]]

In [ ]:
# Create a dataframe of some sample customer complaints
import pandas as pd
df_custom = pd.DataFrame({'complaints': ["I can not get from chase who services my mortgage, who owns it and who has original loan docs",
                                  "The bill amount of my credit card was debited twice. Please look into the matter and resolve at the earliest.",
                                  "I want to open a salary account at your downtown branch. Please provide me the procedure.",
                                  "unwanted service activated and money deducted automatically ",
                                  "How can I know my CIBIL score?",
                                  "Where are the bank branches in the city of Patna?"]})
df_custom

In [ ]:
# Create a new column of predicted topics of each complaint, predicted using the tuned Logistic Regression model
df_custom['Predicted Category'] = df_custom['complaints'].apply(lambda x: topic_predictor([x]))
df_custom

### ***We conclude that the Logistic Regression model is best for making a prediction on custom complaints data.***

In [ ]:
!pip install gradio
import gradio as gr

# Function to predict a topic for custom text

def topic_predictor(text):

    target_names = ["Bank Account services", "Credit card or prepaid card", "Others", "Theft/Dispute Reporting", "Mortgage/Loan"]

    complaint_text = clean_texts(text)
    complaint_text = lemma_texts(complaint_text)
    complaint_text = singular_nouns(complaint_text)
    X_new_count = count_vector.transform([complaint_text])
    X_new_tfidf = tfidf_transform.transform(X_new_count)
    prediction = lr.predict(X_new_tfidf)

    return target_names[prediction[0]]


# Create a Gradio Interface
interface = gr.Interface(
    fn=topic_predictor,  # Function for prediction
    inputs="text",       # Input type: text
    outputs="text",      # Output type: text
    title="Automatic Ticket Classification Model",
    description="Enter a text and the model will automatically classify the category of complaints.",
    theme=gr.themes.Base(
        primary_hue="blue",          # Primary button and accent color
        secondary_hue="gray",        # Secondary elements
        neutral_hue="indigo",        # Neutral elements (e.g., borders)
        text_size="lg"               # Text size
    )
)

# Launch the app
interface.launch()